In [14]:
# import some useful libraries
import glob, nltk, joblib
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import metrics

from PIL import Image
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer

from music21 import midi
from plugins.midi2img import midi2img
from plugins.img2midi import img2midi
from IPython.display import clear_output

In [15]:
DATA_PATH = '../data/dataset'

In [16]:
# load in the metadata
# create a list of captions that concatenate the piece description and arousal
# lower case te caption list
midi_meta = pd.read_csv(f'{DATA_PATH}/commu_meta_desc.csv')
# midi_meta['caption_list'] = midi_meta['piece_description'].str.lower()+ ". " + midi_meta['piece_arousal'].str.lower()
midi_meta

,Unnamed: 0,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,description,description_list,caption_list
0,342,"['c', ' major']","[['Dm7', 'Dm7', 'Dm7', 'Dm7', 'Gsus4', 'Gsus4'...",['mid high'],8,65,new age,['main melody'],['acoustic piano'],standard,4-Apr,33,40,train,commu00343,"""Whispers of Lucid Skies""\nTraversing ethereal...",whispers of lucid skies traversing ethereal re...,"c, major, mid high, main melody, new age, aco..."
1,7279,"['c', ' major']","[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'F',...",['mid high'],8,90,cinematic,['main melody'],['vibraphone'],standard,4-Apr,56,70,train,commu07280,"""An enchanting symphony of soaring dreams, whe...",an enchanting symphony of soaring dreams where...,"c, major, mid high, main melody, cinematic, v..."
2,2734,"['c', ' major']","[['C', 'C', 'C', 'C', 'G', 'G', 'G', 'G', 'Am7...",['mid'],4,100,new age,['pad'],['acoustic piano'],standard,4-Apr,33,42,train,commu02735,"""Uncharted Melody: A harmonious journey throug...",uncharted melody a harmonious journey through ...,"c, major, mid, pad, new age, acoustic piano"
3,6178,"['a', ' minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",['mid'],8,70,new age,['pad'],['synth pad'],standard,4-Apr,81,82,train,commu06179,"""Whispers of Shadows: A Melody's Dance in the ...",whispers of shadows a melodys dance in the twi...,"a, minor, mid, pad, new age, synth pad"
4,3041,"['a', ' minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",['mid'],8,85,cinematic,['accompaniment'],['string ensemble'],standard,4-Apr,63,124,train,commu03042,"""Whispers of Shadows"" - A haunting symphony, w...",whispers of shadows a haunting symphony where ...,"a, minor, mid, accompaniment, cinematic, stri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,5421,"['c', ' major']","[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm7...",['mid low'],8,50,new age,['pad'],['string ensemble'],standard,4-Apr,81,82,train,commu05422,"""Melodic whispers under a starlit sky, embraci...",melodic whispers under a starlit sky embracing...,"c, major, mid low, pad, new age, string ensemble"
192,3259,"['a', ' minor']","[['Dm7', 'Dm7', 'Dm7', 'Dm7', 'Dm7', 'Dm7', 'D...",['mid high'],8,80,cinematic,['sub melody'],['acoustic piano'],standard,4-Apr,38,56,train,commu03260,"""Whispers in Shadows: An Enchanting Melody Awa...",whispers in shadows an enchanting melody awake...,"a, minor, mid high, sub melody, cinematic, ac..."
193,4716,"['c', ' major']","[['Cmaj7', 'Cmaj7', 'Cmaj7', 'Cmaj7', 'G', 'G'...",['high'],8,80,cinematic,['sub melody'],['acoustic piano'],standard,4-Apr,31,58,train,commu04717,"""Whispers of cinematic dreams unfold, as a you...",whispers of cinematic dreams unfold as a young...,"c, major, high, sub melody, cinematic, acoust..."
194,2651,"['a', ' minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",['mid'],8,120,cinematic,['accompaniment'],['string ensemble'],standard,4-Apr,99,100,train,commu02652,"""Lost in Shadows: The Enigmatic Serenade paint...",lost in shadows the enigmatic serenade paintin...,"a, minor, mid, accompaniment, cinematic, stri..."


In [17]:
# read in music and prepare it in a list of notes
midis = []
for file in tqdm(glob.glob(f"{DATA_PATH}/commu_midi/train/raw/*.mid")):
    midis.append(file)

for file in tqdm(glob.glob(f"{DATA_PATH}/commu_midi/val/raw/*.mid")):
    midis.append(file)

  0%|          | 0/10381 [00:00<?, ?it/s]

  0%|          | 0/763 [00:00<?, ?it/s]

In [8]:
# convert MIDI to image
# declare output path
output_path = f"{DATA_PATH}/commu_img/train_desc"

# instantiate a midi2img converter object
midi2img_obj = midi2img(output_path)

# for each midi file, convert it to image
for midi in tqdm(midis):
    data, metadata = midi2img_obj.convert_to_image(midi)
    for img_key in data.keys():
        try:
            # if img_key[-1:] != "0": continue
            img_path = output_path+"/"+midi.split("/")[-1].replace(".mid",f"_{img_key}.png")
            img = Image.open(img_path)
            basewidth = 106
            hsize = 106
            img = img.resize((basewidth,hsize), Image.ANTIALIAS)
            img.save(img_path)
        except Exception as e:
            print("Error in converting: ", e)
            pass

  0%|          | 0/11144 [00:00<?, ?it/s]

C:\Users\ktrin\AppData\Local\Temp\ipykernel_17100\2840756171.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((basewidth,hsize), Image.ANTIALIAS)


Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00001_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00002_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00003_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00004_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00005_Piano_1.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00006_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00007_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train_desc/raw\\commu00008_Piano

In [18]:
# Dataset Construction
def access_images(path, metadata, piece_name='piece_name', num_images=float("inf")):
    """
        Fetch a defined number of images from a directory
        and return their normalized pixels arrays
        Inputs:
            path: path to the directory containing the images
            img_list: list of image names
            num_images: number of images to fetch
        Output:
            image_arr: pixel array of images
            img: the original image
    """
    pixels = {}

    # shuffle the metadata to get a random sample
    if num_images != float("inf"):
        metadata = metadata.sample(n=num_images).reset_index(drop=True)
    else:
        metadata = metadata.sample(frac=1).reset_index(drop=True)

    i = 0
    # construct a dictionary of empty lists with the song name
    for piece in tqdm(metadata[piece_name].unique()):
        if i >= num_images:
            break
        
        # construct the file path for the image
        file = f"{path}/{piece}_Piano_0.png"
        try:
            # Normalize the image
            img = Image.open(file, 'r')
            img = img.convert('1')
            pix = np.array(img.getdata())
            pix = pix.astype('float32')
            pix /= 255.0

            # pixels.append(pix.reshape(106,106,1))
            # once a valid image is found, break the loop
            if piece not in pixels.keys():

                pixels[piece] = [pix.reshape(106,106,1)]
            else:
                pixels[piece].append(pix.reshape(106,106,1))

        except Exception as e:
            # print("Error in fetching: ", e)
            pass
        i+=1
    return pixels

img_dir = f"{DATA_PATH}/commu_img/train_desc/raw"
# get train data only
# midi_meta = midi_meta[midi_meta['split_data'] == 'train']
pixels_arr = access_images(img_dir, midi_meta, piece_name='id')
len(pixels_arr.keys())

  0%|          | 0/196 [00:00<?, ?it/s]

195

In [19]:
# # Dataset Construction
# def access_images(path, metadata, piece_name='piece_name', num_images=float("inf")):
#     """
#         Fetch a defined number of images from a directory
#         and return their normalized pixels arrays
#         Inputs:
#             path: path to the directory containing the images
#             img_list: list of image names
#             num_images: number of images to fetch
#         Output:
#             image_arr: pixel array of images
#             img: the original image
#     """
#     pixels = {}

#     # shuffle the metadata to get a random sample
#     metadata = metadata.sample(n=num_images).reset_index(drop=True)

#     # fetch all images from the directory
#     i = 0
#     # construct a dictionary of empty lists with the song name
#     for piece in tqdm(metadata[piece_name].unique()):
#         # if row['piece_name'] not in pixels.keys():
#         #     pixels[row['piece_name']] = []
#         if i >= num_images:
#             break

#         for file in (glob.glob(f"{path}/*.png")):
#             try:
#                 img_path = file.split("/")[-1]
#                 # ignore the images that are not the first image
#                 if img_path[-5:-4] != '0':
#                     continue

#                 # Normalize the image
#                 img = Image.open(file, 'r')
#                 img = img.convert('1')
#                 pix = np.array(img.getdata())
#                 pix = pix.astype('float32')
#                 pix /= 255.0

#                 # pixels.append(pix.reshape(106,106,1))
#                 # once a valid image is found, break the loop
#                 if img_path.find(piece) != -1:
#                     if piece not in pixels.keys():
#                         pixels[piece] = [pix.reshape(106,106,1)]
#                     else:
#                         pixels[piece].append(pix.reshape(106,106,1))
#                     break

#             except Exception as e:
#                 print("Error in fetching: ", e)
#                 pass

#         i += 1
#     return pixels


# img_dir = f"{DATA_PATH}/commu_img/train/raw"
# # get train data only
# midi_meta = midi_meta[midi_meta['split_data'] == 'train']
# pixels_arr = access_images(img_dir, midi_meta, piece_name='id', num_images=2048)
# pixels_arr.keys()
# # np.unique(pixels_arr['commu00001'])

In [20]:
# create a dataframe of image arrays and piece names
join_col = 'id'
sound_image = pd.DataFrame(columns=[join_col, 'image_array'])#from_dict(pixels_arr, orient='index').reset_index()
sound_image[join_col] = pixels_arr.keys()
sound_image['image_array'] = pixels_arr.values()
sound_image

# merge with metadata
midi_meta = midi_meta.merge(sound_image, on=join_col, how='left').dropna(subset=[join_col])


# create image-caption pairs
training_set = []
for i, row in midi_meta.iterrows():
    caption = row['description_list']
    images = row['image_array']
    id = row['id']
    try:
        for image in images:
            training_set.append((id, image, caption))
    except:
        pass



In [21]:
# save data as joblib
joblib.dump(training_set, f'{DATA_PATH}/training_set_desc.joblib')

['../data/dataset/training_set_desc.joblib']